In [2]:
import numpy as np
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import simpledialog

In [7]:
img = Image.open("sample.jpg")
x,y = img.size
print(f"Total Pixels for The Given Image: {x*y}")

Total Pixels for The Given Image: 60217344


In [15]:
def reveal_pixels(image_path, pixels_to_reveal):
    """
    Reveal a specified number of pixels in an image.
    
    Args:
    image_path (str): Path to the image file.
    pixels_to_reveal (int): Number of pixels to reveal.
    
    Returns:
    Image: Image with the specified number of pixels revealed.
    """
    img = Image.open("sample.jpg")
    img_array = np.array(img)
    
    total_pixels = img_array.shape[0] * img_array.shape[1]
    mask = np.zeros(total_pixels, dtype=bool)
    mask[:pixels_to_reveal] = True
    np.random.shuffle(mask)
    mask = mask.reshape(img_array.shape[:2])

    for i in range(img_array.shape[2]):
        img_array[:, :, i] = img_array[:, :, i] * mask

    new_img = Image.fromarray(img_array)
    return new_img

def display_image(image):
    """
    Display an image in a new window, scaled to fit the window.
    
    Args:
    image: Image object to display.
    """
    root = tk.Toplevel()
    
    # Get the screen width and height
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    
    # Resize the image to fit the window
    img_width, img_height = image.size
    if img_width > screen_width or img_height > screen_height:
        # Scale down the image if it's larger than the screen
        scale_factor = min(screen_width / img_width, screen_height / img_height)
        new_width = int(img_width * scale_factor)
        new_height = int(img_height * scale_factor)
        image = image.resize((new_width, new_height))
    
    # Convert the image to PhotoImage format
    imgtk = ImageTk.PhotoImage(image=image)
    
    # Create a label to display the image
    label = tk.Label(root, image=imgtk)
    label.image = imgtk  # Keep a reference, otherwise it's garbage collected.
    label.pack()
    
    root.mainloop()

def ask_pixels_and_display():
    """
    Ask the user for the number of pixels to reveal and display the modified image.
    """
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    image_path = 'sample.jpg'  # Replace with your image file path
    pixels_to_reveal = simpledialog.askinteger("Input", "How many pixels to display?",
                                               parent=root, minvalue=0, maxvalue=90000000)
    root.destroy()  # Close the root window
    
    if pixels_to_reveal is not None:  # Check if input was given
        revealed_image = reveal_pixels(image_path, pixels_to_reveal)
        display_image(revealed_image)
        
ask_pixels_and_display()        